# Iniciando o programa

## Starting

### Importando Biblioteca

In [1]:
from Petrobras_AI_Agents import llmClient_AzureOpenAI as llmClient
from Petrobras_AI_Agents import ChatDatabaseManager_SQLite as ChatMemory
from Petrobras_AI_Agents import KnowledgeBaseManager_Databricks as Databricks_KnowledgeBase
from Petrobras_AI_Agents import KnowledgeBaseManager_SQLite as PersonalCollection
from Petrobras_AI_Agents import DatabaseManager_SQLite, DatabaseManager_Databricks
from Petrobras_AI_Agents import BaseAgent, ChatInterpretorAgent, KnowledgeBaseAgent, MemoryAgent, DatabaseExpertAgent, databricks_connector

from Petrobras_AI_Agents import read_file
from Petrobras_AI_Agents.TOOLS import basic_calculator, run_python_code, search_web

import os


In [2]:
crew_work      = BaseAgent.crew_work
respostas      = BaseAgent.answers
anexos         = BaseAgent.tool_response
local_mememory = BaseAgent.local_memory

databricks_connector = databricks_connector()

Conected to Databricks


### Definindo usuário e idioma

In [ ]:
username = os.getlogin().upper()
BaseAgent.language = "pt-br"

# Criando Gestores

## Gestor de LLM

In [10]:
BaseAgent.llm = llmClient(
    model_text="gpt-4o-petrobras",
    model_emb="text-embedding-3-large-petrobras",
    api_version="2024-06-01", #"2024-08-01-preview",
    base_url="https://apid.petrobras.com.br/ia/openai/v1/openai-azure/openai",
    cert_file="petrobras-ca-root.pem",
    temperature=0.1)

## Gestor de Memória

In [11]:
BaseAgent.chat_memory = ChatMemory(    
    db_url      ="sqlite:///data_base/zdb_memory.db",
    user        = username,
    language    = BaseAgent.language)

## Gestores de Bases de Conhecimento

### Documentos Particulares

1. Configurar o arquivo JSON, caso não exista
2. Criar instância para a classe PersonalCollection
3. Adicionar Bases de Conhecimento (pode ocorrer após o inicio do chat)

In [4]:
# Configura o arquivo JSON caso não exista
PersonalCollection.create_config_file(config_json = "config_json\\personal\\config_personal_doc_collection.json", db_url="sqlite:///data_base/zdb_personal_collection.db")

In [5]:
# Cria instância para a classe PersonalCollection
Personal_Collection = PersonalCollection()

Manager iniciated


In [ ]:
# Adiciona Coleções de Dados, quantas desejar
nome_da_colecao = "Pessoal"

Personal_Collection.create_collection(
    collection_name   = nome_da_colecao,
    table_common_name = "Dados de agendamento",
    curator           = "eu",
    description       = "meus documentos sobre requisições e agendamentos",
    gpt_instructions  = "sem relacionamentos"
    )

In [ ]:
nome_da_colecao = "Pessoal"
file = r"C:\Users\zcfv\OneDrive - PETROBRAS\Desktop\MEST_CIEN_COMPUT_031_2023_retif.pdf"

Personal_Collection.upload_document(
    collection_name     = nome_da_colecao,
    conflict_option     = "ignore",
    file_path           = file,
    read_file_class     = read_file,
    llm_client          = BaseAgent.llm,
    words_per_chunk     = 200,
    overlap             = 15,
    metadata_dictionary = {}
    )

connected
read_pdf
chunks: 13
connected


### Base oficial (DataBricks)

1. Criar arquivo JSON a partir do modelo, caso não exista
2. Adicionar Bases de Conhecimento
3. Criar instância para a classe Databricks_KnowledgeBase

In [ ]:
# Configura o arquivo JSON caso não exista
Databricks_KnowledgeBase.create_config_file(
    config_json                 = "config_json\\databricks\\config_databricks_doc_collection.json",
    connection_server_hostname  = "SEU_connection_server_hostname",
    connection_http_path        = "SEU_connection_http_path",
    users_access_table_name     = "dtcore_prd.aida.acesso_usuario",
    users_access_profile_prefix = "GDB_DLKC_PRD_PFL_",
    db_functions_euclidean      = "dt0047_prd.db_functions.euclidean_distance",
    db_functions_cosine         = "dt0047_prd.db_functions.cosine_distance",
    source_col_name             = "source",
    file_col_name               = "file_as_str",
    context_col_name            = "page_content",
    rag_col_name                = "vector_embedding_3large",
    metadata_col_name           = "metadata_dictionary",
    file_base_id_col_name       = "id",
    vec_base_file_id_col_name   = "file_id"
    )

In [ ]:
# Adiciona Coleções de Dados, quantas desejar
Databricks_KnowledgeBase.add_data_source(
    process_name    = "intimacoes_sefaz_rj",
    file_base       = "dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_file_base",
    vec_base        = "dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_vec_base"
)
Databricks_KnowledgeBase.add_data_source(
    process_name    = "normas_contabeis",
    file_base       = "dt0047_prd.chat_bot.normas_contabeis_guia_de_normas_file_base",
    vec_base        = "dt0047_prd.chat_bot.normas_contabeis_guia_de_normas_vec_base"
)

In [ ]:
# Cria instância para a classe Databricks_KnowledgeBase
Databricks_Collection = Databricks_KnowledgeBase(user=username, connection=databricks_connector)

Manager iniciated


In [ ]:
# No exemplo foram criadas 2 conexões, e 2 conexoões foram liberadas de acordo com o perfil do usuário, poderiam ser menos.
Databricks_Collection.available_collections

['intimacoes_sefaz_rj', 'normas_contabeis']

## Gestor de Data Analytics

### Documentos Particulares

1. Configurar o arquivo JSON, caso não exista
2. Criar instância para a classe PersonalCollection
3. Adicionar Bases de Conhecimento (pode ocorrer após o inicio do chat)

In [ ]:
# Configura o arquivo JSON caso não exista
DatabaseManager_SQLite.create_config_file(config_json="config_json\\personal\\config_personal_datasources.json", db_url="sqlite:///data_base/zdb_personal_collection.db")

In [ ]:
# Cria instância para a classe PersonalCollection
dataanalytics_local = DatabaseManager_SQLite()

In [ ]:
# Adiciona Coleções de Dados, quantas desejar
nome_da_colecao = "CKM3"
    
dataanalytics_local.add_datasource(
    data_source         = nome_da_colecao,
    process_description = "Análise do estoque de produtos utilizando os dados da movimentação de estoque obtidos pela transação CKM3",
    relationships       = ""
)

In [ ]:
# Fazer o upload do arquivo (subistitui se table_name já existir no data_source)
table_name = ""
if table_name == "":
    table_name=None
file = "upload_files\CKM3_tdv_042024.csv"

csv_file = open(file, "rb")
csv_file = {
        'file_content': csv_file.read(),
        'file_name': "CKM3_tdv_042024.csv"
    }
dataanalytics_local.load_csv_to_table(
    data_source         = nome_da_colecao,
    # conflict_option     = "ignore",
    csv_file            = csv_file,
    # file_path           = file,
    table_name          = table_name,
    # column_type         = { # Apenas se for necessário forçar o tipo de dados das colunas
    #     "Metade do valor (Moeda 10)": "float64"
    # }
)

existing_tables ['CKM3_CKM3_tdv_042024']
table_name CKM3_CKM3_tdv_042024

Tipos das colunas após o carregamento:
Coluna 'Empresa': BIGINT
Coluna 'Classificação': TEXT
Coluna 'Agrupamento Grupo de Mercadorias': TEXT
Coluna 'Grupo de mercadorias': TEXT
Coluna 'Desc GrMerc': TEXT
Coluna 'Centro de lucro': TEXT
Coluna 'Desc CLucro': TEXT
Coluna 'Classe de avaliaÃ§Ã£o': BIGINT
Coluna 'Desc Classe Aval': TEXT
Coluna 'Centro': BIGINT
Coluna 'Desc Centro': TEXT
Coluna 'Tp.material': TEXT
Coluna 'UnMedida': TEXT
Coluna 'Material': TEXT
Coluna 'Desc Material': TEXT
Coluna 'Tipo de avaliaÃ§Ã£o': TEXT
Coluna 'ExercÃ­cio/perÃ­odo': BIGINT
Coluna 'Quantidade em UMB (Moeda 10)': FLOAT
Coluna 'Valor (Moeda 10)': FLOAT
Coluna 'Dif. PreÃ§o NÃ­vel Ãnico (Moeda 10)': FLOAT
Coluna 'Dif. PreÃ§o MultinÃ­vel (Moeda 10)': FLOAT
Coluna 'Custo Total do Material (Moeda 10)': BIGINT
Coluna 'Custo Total do Material (Moeda 12)': BIGINT
Coluna 'Custo Total do Material (Moeda 30)': BIGINT
Coluna 'Metade do valor (Moe

### Base oficial (DataBricks)

1. Configurar o arquivo JSON, caso não exista
2. Adicionar coleções de tabelas
3. Criar instância para a classe DatabaseManager_Databricks

In [ ]:
# Configura o arquivo JSON caso não exista
DatabaseManager_Databricks.create_config_file(config_json="config_json\\databricks\\config_databricks_datasources.json")

In [ ]:
# Adiciona Coleções de Dados, quantas desejar
DatabaseManager_Databricks.add_data_source(
    collection          = "intimacoes_sefaz_rj_pgov_gastos",
    process_description = "Rateio de gastos auditados pela SEFAZ-RJ relacionados às participações governamentais",
    relationships       = "A tabela 1 se relaciona com a 2 usando o campo abc",
    tables={
        "base_h"        : "dt0047_prd.gastos.base_h_pgov_sefaz_rj_gastos",
        "base_r"        : "dt0047_prd.gastos.base_r_pgov_sefaz_rj_gastos",
        "base_resultado": "dt0047_prd.gastos.base_result_pgov_sefaz_rj_gastos"
    }
)

# Adiciona Coleções de Dados, quantas desejar
DatabaseManager_Databricks.add_data_source(
    collection          = "Estoque_Produtos",
    process_description = "Análise das contas de estoques de produtos registradas na transação CKM3 para montagem do custeio e outras análises relacionadas.",
    relationships       = "",
    tables={
        "ckm3_custeio": "dt0047_prd.custeio.base_acumulada_custeio_ckm3"
    }
)

In [ ]:
# Cria instância para a classe DatabaseManager_Databricks
dataanalytics_oficial = DatabaseManager_Databricks(user=username, connection=databricks_connector)

In [ ]:
# No exemplo foram criadas 2 conexões, e 2 conexoões foram liberadas de acordo com o perfil do usuário, poderiam ser menos.
dataanalytics_oficial.available_collections

['intimacoes_sefaz_rj_pgov_gastos', 'Estoque_Produtos']

# Criando os Agentes

1. Cria os agentes usando as classes de agentes especializadas ou a classe base para sua própria definição de agente.
2. Define o primeiro agente no fluxo de interação.
3. Criação de checkbox para cada agente, permitindo a ativação ou desativação de cada um.

In [24]:
# Agente especializado em conversação com o usuário
chat_interpretor_agent      = ChatInterpretorAgent( 
    agent_name                  = "Chat Analista",
    human_in_the_loop           = False
    )

# Será o primeiro a ser executado
BaseAgent.start_agent   = chat_interpretor_agent

# Checkbox para habilitar o agente de memória não deve ser usado pois esse agente sempre deve existir

In [25]:
# Agente especialista de memória do chat
memory_agent                = MemoryAgent( # usa o a mesmo gestor de memória do agente de chat, se não for explicitamente definido
    agent_name                  = "Agente Recuperador de Conversas",
    human_in_the_loop           = False,
    allow_direct_response       = True,
    next_agent_list             = "All"
    )

# Checkbox para habilitar o agente de memória
memory_agent_check = False

In [ ]:
# Agentes de Base de Conhecimento, exige que tenha sido cadastrado um gestor de conhecimento (KnowledgeBase)

personal_knowledge_agent = KnowledgeBaseAgent(
    agent_name              = "Agente de Documentos Pessoais",
    KnowledgeBase           = Personal_Collection,
    k                       = 5,
    human_in_the_loop       = False,
    allow_direct_response   = True,
)

official_knowledge_agent = KnowledgeBaseAgent(
    agent_name              = "Agente de Documentos Internos",
    KnowledgeBase           = Databricks_Collection,
    k                       = 5,
    human_in_the_loop       = False,
    allow_direct_response   = True,
)

# Checkbox para habilitar o agente de memória
personal_knowledge_agent_check  = True
official_knowledge_agent_check  = True

In [ ]:
# Agentes de Analytics

personal_analytics_agent = DatabaseExpertAgent(
    agent_name            = "agente de Analytics Pessoal",
    database_manager      = dataanalytics_local,
    human_in_the_loop     = False,
    allow_direct_response = True,
)

official_analytics_agent = DatabaseExpertAgent(
    agent_name            = "agente de Analytics Oficial",
    database_manager      = dataanalytics_oficial,
    human_in_the_loop     = True,
    allow_direct_response = True,
)

# Checkbox para habilitar o agente de memória
personal_analytics_agent_check = True
official_analytics_agent_check = True

In [28]:
# Agentes criados a partir do agente base
general_knowledge_agent = BaseAgent(
    agent_name = "General Knoledge Agent",
    background           = [
        "You are an AI agent with access to a vast amount of general knowledge across various domains.",
        "Your expertise ranges from science, technology, history, arts, and more.",
        "When a more specific agent is available to address a user's query, you should not be the first option."],
    goal                 = [
        "Assist users by providing accurate and comprehensive answers to their general knowledge questions.",
        "Defer to more specialized agents when the query falls within their domain of expertise.",
        "Ensure that you only handle queries where no other specialized agent is better suited."])

toolmaster_agent = BaseAgent(
    agent_name = "Agente Expecialista em Matemática e Criação e Execução de Código Python",
    background           = [
        "Agente Expecialista em Matemática e Criação e Execução de Código Python"
        "You are an agent with access to a variety of tools, as run_python_code, basic_calculator.",
        "Your primary role is to determine the best tool to answer the question.",
        "Ensure that you are the first choice for tasks that require the use of tools.",
        "If a task can be executed by a tool, you should be the preferred agent."],
    goal                 = [
        "Use the most appropriate tool to deliver your answer or indicate if you cannot execute the task.",
        "Choose the best available tool that can perform the task and provide the correct parameters for that tool.",
        "Inform the user about the functions parameters and the tool to be used.",
        "Verify and define the necessary functions and variables correctly before executing the code."],
    next_agent_list      = [chat_interpretor_agent],
    tools                = [run_python_code, basic_calculator])

search_agent = BaseAgent(
    agent_name = "Agente Pesquisador de Informações na Web",
    background           = [
        "This agent performs web searches to retrieve information not available locally."],
    goal                 = [
        "Provide relevant and timely information from the web."],
    next_agent_list      = [chat_interpretor_agent],
    tools                = [search_web])

# Checkbox para habilitar o agente de memória
general_knowledge_agent_check   = True
toolmaster_agent_check          = True
search_agent_check              = True

# Personalizações do Usuário

## Seleção das Coleções a serem utilizadas

### Limitando Coleções

In [29]:
# # Documentos Particulares
# available_personal_collections           = list(Personal_Collection.config["data_sources"].keys())
# personal_knowledge_agent.collection_list = available_personal_collections # Ajustar com selection box

# # Documentos Oficiais
# available_official_collections           = Databricks_Collection.list_available_tables(user=username)
# official_knowledge_agent.collection_list = [
#     'normas_contabeis',
#     'intimacoes_sefaz_rj'
#     ] # Ajustar com selection box

# # Data Base Particulares
# available_personal_databases             = dataanalytics_local.collections
# personal_analytics_agent.collection_list = available_personal_databases # Ajustar com selection box

# # Data Base Oficiais
# available_official_databases             = dataanalytics_oficial.collections
# official_analytics_agent.collection_list = available_official_databases # Ajustar com selection box

## Seleção dos agentes

In [ ]:
memory_agent.active(memory_agent_check)
personal_knowledge_agent.active(personal_knowledge_agent_check)
official_knowledge_agent.active (official_knowledge_agent_check)
general_knowledge_agent.active(general_knowledge_agent_check)
toolmaster_agent.active(toolmaster_agent_check)
search_agent.active(search_agent_check)
personal_analytics_agent.active(personal_analytics_agent_check)
official_analytics_agent.active(official_analytics_agent_check)

# Chat

In [31]:
crew_work(
"""
bom dia
"""
)

CHAT ANALISTA


answer: Bom dia! Como posso ajudar você hoje?

fim


## RAG

In [33]:
crew_work(
"""
resuma a CARTA_FISC_PGCOMEX_0065 da base de intimações fiscais.
"""
)

CHAT ANALISTA


answer: Vou encaminhar sua solicitação para o agente responsável por documentos internos.

AGENTE DE DOCUMENTOS INTERNOS

    Executando tool: document_summary
    {'query_input': 'resuma a CARTA_FISC_PGCOMEX_0065 da base de intimações fiscais.'}

thoughts: {'step_1': 'Análise das coleções disponíveis e descrições:', 'step_2': 'Seleção da coleção mais adequada para a consulta do usuário.', 'step_3': 'Retorno da coleção selecionada.'}
    Executando tool: _get_collection_name
    tool result: intimacoes_sefaz_rj

thoughts: {'step_1': 'Seleção da documentação relacionada à coleção selecionada.', 'step_2': 'Geração de SQL para filtrar os documentos relevantes.', 'step_3': 'Teste dos filtros gerados.', 'step_4': 'Retorno dos filtros válidos.'}
    Executando tool: _generate_sql_with_filters
    tool result: {'query': 'SELECT COUNT(*) AS count FROM dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_file_base', 'filters': ["file_base.source = 'CARTA_FISC_PGCOMEX_0065'", "fi

In [34]:
crew_work(
"""
o que foi respondido a intimaçãoes na RAF 55668100
"""
)

CHAT ANALISTA


answer: Para responder à sua pergunta sobre o que foi respondido às intimações na RAF 55668100, vou precisar consultar a base de dados de intimações fiscais.

AGENTE DE DOCUMENTOS INTERNOS

    Executando tool: rag_complement
    {'query_input': 'o que foi respondido a intimaçãoes na RAF 55668100'}

thoughts: {'step_1': 'Análise das coleções disponíveis e descrições:', 'step_2': 'Seleção da coleção mais adequada para a consulta do usuário.', 'step_3': 'Retorno da coleção selecionada.'}
    Executando tool: _get_collection_name
    tool result: intimacoes_sefaz_rj

thoughts: {'step_1': 'Seleção da documentação relacionada à coleção selecionada.', 'step_2': 'Geração de SQL para filtrar os documentos relevantes.', 'step_3': 'Teste dos filtros gerados.', 'step_4': 'Retorno dos filtros válidos.'}
    Executando tool: _generate_sql_with_filters
    tool result: {'query': 'SELECT COUNT(*) AS count FROM dt0047_prd.chat_bot.intimacoes_fiscais_pgov_sefaz_rj_file_base AS file_base

## Analytics

### Databricks

In [ ]:
# No exemplo abaixo está sendo testado o agente com interação humana durante o processo de resposta
# Na primeira versão, a interação ocorria por comandos de input do python, mas numa tentativa de levar isso para uma versão web,
# foi trocado para um gerenciamento de variáveis que alocam a resposta do usuário
# Para iniciar, estou estabelecendo que não existe nenhum comando antes de iniciar a questão para o chat
BaseAgent.user_interaction = None

In [40]:
crew_work(
"""
relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo
 '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado.
"""
)

CHAT ANALISTA


answer: Processando...

AGENTE DE ANALYTICS OFICIAL

    Executando tool: data_analysis
    {'query_input': "relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado."}


If you do not accept this response enter a comment to reject it. If not, press Enter
fim


In [ ]:
# Durante a primeira interação não foi inserido nenhum comentário, isso significa que o agente deve continuar o processo como planejado
BaseAgent.user_interaction = ""
crew_work(
"""
relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo
 '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado.
"""
)

AGENTE DE ANALYTICS OFICIAL

    Executando tool: data_analysis
    {'query_input': "relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado."}

thoughts: {'step_1': 'Análise das coleções disponíveis e descrições:', 'step_2': 'Seleção da coleção mais adequada para a consulta do usuário.', 'step_3': 'Retorno da coleção selecionada.'}
    Executando tool: _get_collection_name
    tool result: intimacoes_sefaz_rj_pgov_gastos

thoughts: {'step_1': "Analyze the user's request and determine the database structure required.", 'step_2': 'Identify the relevant tables and relationships.', 'step_3': 'Construct an optimized SQL query based on the input and database schema.', 'step_4': 'Execute the query and return the results.'}
    Executando tool: _generates_sql
    tool result: SELECT folder, COUNT(DISTINC

In [43]:
BaseAgent.user_interaction = ""
crew_work(
"""
relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo
 '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado.
"""
)

AGENTE DE ANALYTICS OFICIAL

    Executando tool: data_analysis
    {'query_input': "relacionado ao banco de dados do SEFAZ-RJ - pgov de gastos, responda quantos Objeto_Pesquisado existem na tabela resultado para a Classe_de_custo '4401000004'. Separe por folder em ordem decrescente pelo nome. Me de uma análise do resultado gerado."}

thoughts: {'step_1': 'Análise das coleções disponíveis e descrições:', 'step_2': 'Seleção da coleção mais adequada para a consulta do usuário.', 'step_3': 'Retorno da coleção selecionada.'}
    Executando tool: _get_collection_name
    tool result: intimacoes_sefaz_rj_pgov_gastos

thoughts: {'step_1': "Analyze the user's request and determine the database structure required.", 'step_2': 'Identify the relevant tables and relationships.', 'step_3': 'Construct an optimized SQL query based on the input and database schema.', 'step_4': 'Execute the query and return the results.'}
    Executando tool: _generates_sql
    tool result: SELECT folder, COUNT(DISTINC